# Reflectancia del água y clorofila

En este notebook explorará el comportamiento de la reflectancia saliente del agua al variar la concentración de clorofila.

## Introduccion a python

Si nunca uso un notebook de jupyter es muy sencillo. Hay celdas con texto, como esta, y celdas con codigo, como la de abajo.

Para ejecutar una celda con código haga click en ella y luego en `>|run` en la barra de herramientas o `ctrl+enter`.

In [ ]:
2+2

La siguiente celda es importante, ya que carga todas las herramientas que utilizará. Haga click en `>|run` o `ctrl+enter`

In [ ]:
from functions import *

## Agua y clorofila

### Actividad 1: explore la firma espectral para distintas concentraciones de clorofila.

In [ ]:
interact(plt_f, chl = widgets.BoundedFloatText(value=5, min=0.001, max=100, description="Chla [mg/m3]"), 
                sed = fixed(0), data=fixed(None), compute=fixed(False), text=fixed(False),
                satellite=fixed(None));

### Actividad 2 : encuentre que contenido de clorofila reproduce los datos CZCS

In [ ]:
interact(plt_f, chl = widgets.BoundedFloatText(value=5, min=0.001, max=100, description="Chla [mg/m3]"), 
                sed=fixed(0),data=fixed(None), compute=fixed("data"),text=fixed(False),
                satellite=fixed("czcs"));

### Actividad 3: encuentre que contenido de clorofila reproduce los datos de seawifs

In [ ]:
interact(plt_f, chl = widgets.BoundedFloatText(value=5, min=0.001, max=100, description="Chla [mg/m3]"), 
                sed = fixed(0), data=fixed(None), compute=fixed("data"), text=fixed(False),
                        satellite=widgets.Dropdown(options=list_satellite.keys(), description="Satélite"));

### Actividad 4: encuentre que contenido de clorofila que reproduce los datos de seawifs

In [ ]:
interact(plt_f, chl = widgets.BoundedFloatText(value=5, min=0.001, max=100, description="Chla [mg/m3]"), 
                sed = fixed(0), data=widgets.Dropdown(options=list_data.keys(),description="Datos"),
                text=fixed(False), compute=fixed("data"), satellite=fixed(None));

### Adicional 1: Puede ahora trabajar tanto con la variable concentración de clorofila como concentración de sedimientos

In [ ]:
interact(plt_f, chl = widgets.BoundedFloatText(value=5, min=0.001, max=100, description="Chla [mg/m3]"), 
                sed = widgets.BoundedFloatText(value=0, min=0.0, max=1e3, description="Sed [mg/l]"), 
                data=fixed(None), compute=widgets.Checkbox(description="Mostrar reflectancias"),
                text=widgets.Checkbox(description="Imprimir reflectancias"),
                satellite=widgets.Dropdown(options=list_satellite.keys(), description="Satélite"));